In [1]:
import pandas as pd
import numpy as np 
from functions.Strategy import *
from functions.UtilsCreateDataFrame import *
from functions.UtilsGoogleDrive import *
import time
import joblib

drive = True

C:\Users\giova\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\giova\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
path = "H:\AllTickers_klines_2020-12_2021-12.csv.gz"
coin_df = loadCleanDataFrame(path)
#let's test two months
coin_df

,ETH,BNB,XRP,BTC,ADA,SOL,DOGE,TRX,BUSD,MATIC,LTC,DOT,USDC
time,,,,,,,,,,,,,
2020-12-01 00:00:59.999000+00:00,616.280,31.49085,0.665205,19705.900,0.172355,1.9665,0.003559,0.032380,0.99960,0.019620,87.735,5.37605,0.99960
2020-12-01 00:01:59.999000+00:00,615.520,31.48720,0.665410,19692.585,0.172525,1.9653,0.003560,0.032385,0.99960,0.019590,87.720,5.36925,0.99955
2020-12-01 00:02:59.999000+00:00,616.095,31.58415,0.665940,19682.370,0.173210,1.9655,0.003566,0.032415,0.99965,0.019595,87.735,5.37255,0.99955
2020-12-01 00:03:59.999000+00:00,616.945,31.67950,0.667905,19673.010,0.174180,1.9706,0.003572,0.032495,0.99970,0.019630,87.730,5.37720,0.99955
2020-12-01 00:04:59.999000+00:00,615.795,31.65380,0.666910,19626.910,0.173875,1.9633,0.003569,0.032500,0.99960,0.019590,87.540,5.37010,0.99950
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 23:55:59.999000+00:00,3678.005,511.50000,0.831300,46269.240,1.309000,170.2850,0.170300,0.075420,0.99995,2.521500,146.250,26.69000,0.99990
2021-12-31 23:56:59.999000+00:00,3676.865,511.60000,0.830750,46251.335,1.309000,170.2050,0.170250,0.075410,0.99990,2.523000,146.300,26.69000,0.99995
2021-12-31 23:57:59.999000+00:00,3677.270,511.65000,0.830200,46249.175,1.309500,170.1800,0.170350,0.075440,0.99990,2.525500,146.400,26.69000,1.00000


In [3]:
%%time
if drive: 
    id_ = "1Vg9w6RpPjukasvRqxM4cqPxabDi9MeyS"
    asset_name_1 = "ETH"
    asset_name_2 = "XRP"
    folder_name, id_pair = CreatePairFolder(id_, (asset_name_1, asset_name_2))

cal_w1 = {"hours":12}
beta_table, path_beta = create_beta_table(
    coin_df = coin_df.iloc[:44000*2,:],
    asset_name_1 = asset_name_1,
    asset_name_2 = asset_name_2,
    calibration_window = cal_w1,
    frequency={"minutes": 1},
    safe_output_csv=True,
    n_job=4,
    output_folder = "H:",
    stat_test = "adfuller"
)

if drive:
    UploadFile(file =  path_beta, folder_id = id_pair )

Wall time: 12min 3s


In [14]:
%%time 

tr_w6 = {"hours":12}
tr_w5 = {"hours":6}
tr_w4 = {"hours" : 4}
tr_w3 = {"hours":2}
tr_w2 = {"hours":1}
tr_w1 = {"minutes":30}
tresh = [tr_w1, 
        tr_w2, 
        tr_w3, 
        tr_w4, tr_w5, 
        tr_w6]

result = getCombRet(
    coin_df = coin_df.iloc[:44000*2,:],
    asset_name_1 = asset_name_1,
    asset_name_2 = asset_name_2,
    trading_windows = tresh,
    calib_window = cal_w1,
    p_values = [0.15, 0.1, 0.05, 0.01],
    stop_loss=0.1,
    stat_test = "adfuller",
    safe_beta_csv=True,
    input_folder = "H:",
    output_folder_beta = "H:"
)


path_result = f"H:/{folder_name}.joblib"
joblib.dump(result, path_result)
UploadFile(file =  path_result, folder_id = id_pair )

C:\Users\giova\StatisticalArbitrage\functions\Strategy.py:128: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  coin_df["Month"] = coin_df.index.to_period("M")


Beta Table Loaded


C:\Users\giova\StatisticalArbitrage\functions\Strategy.py:143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  beta_df["Month"] = beta_df.index.to_period("M")


Wall time: 6min


In [15]:
result["2021-01"][0]

,0.15,0.10,0.05,0.01
0 days 00:30:00,0.007909,0.067293,0.49962,0.77815
0 days 01:00:00,0.000135,0.16356,0.412523,0.819706
0 days 02:00:00,0.000016,0.002644,0.013924,0.018016
0 days 04:00:00,0.000003,0.000104,0.002269,0.005712
0 days 06:00:00,0.0,0.000001,0.000018,0.001481
0 days 12:00:00,0.706966,0.0,0.0,0.000326
